# Import data to Sedona and create Polaris SQL queries

In [2]:
import xarray as xr
import os

data_dir = "/home/uribe055/sedona_experiments/processed_data/"

### Code for when data is split into multiple (daily) files

In [3]:
from sedona.spark import SedonaContext
from pyspark.sql.functions import expr

# Start Spark + Sedona
config = SedonaContext.builder(). \
    config('spark.jars.repositories', 'https://artifacts.unidata.ucar.edu/repository/unidata-all/'). \
    config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-3.5_2.12:1.7.1,'
           'org.datasyslab:geotools-wrapper:1.7.1-28.5,'
           'edu.ucar:cdm-core:5.4.2'). \
    config('spark.driver.memory', '10g'). \
    config('spark.driver.maxResultSize', '5g'). \
    config('spark.network.timeout', '1000s'). \
    getOrCreate()
sedona = SedonaContext.create(config)


25/05/25 16:10:08 WARN UDTRegistration: Cannot register UDT for org.geotools.coverage.grid.GridCoverage2D, which is already registered.
25/05/25 16:10:08 WARN SimpleFunctionRegistry: The function rs_union_aggr replaced a previously registered function.
25/05/25 16:10:08 WARN UDTRegistration: Cannot register UDT for org.locationtech.jts.geom.Geometry, which is already registered.
25/05/25 16:10:08 WARN UDTRegistration: Cannot register UDT for org.apache.sedona.common.geometryObjects.Geography, which is already registered.
25/05/25 16:10:08 WARN UDTRegistration: Cannot register UDT for org.locationtech.jts.index.SpatialIndex, which is already registered.
25/05/25 16:10:08 WARN SimpleFunctionRegistry: The function st_envelope_aggr replaced a previously registered function.
25/05/25 16:10:08 WARN SimpleFunctionRegistry: The function st_intersection_aggr replaced a previously registered function.
25/05/25 16:10:08 WARN SimpleFunctionRegistry: The function st_union_aggr replaced a previously

In [20]:
# Read in parquet files as Spark DataFrame
sdf = sedona.read.parquet(f"/home/uribe055/sedona_experiments/data/t2m_202*-*-*.parquet")

# Create a geometry column for spatial queries
sdf = sdf.withColumn("geom", expr("ST_Point(cast(longitude as double), cast(latitude as double))"))
sdf.show()

+--------+---------+------+-------------------+------+--------------+-----------------+
|latitude|longitude|number|               time|expver|2m_temperature|             geom|
+--------+---------+------+-------------------+------+--------------+-----------------+
|    84.0|    -75.0|     0|2023-12-27 00:00:00|  0001|     243.76312|   POINT (-75 84)|
|    84.0|   -74.75|     0|2023-12-27 00:00:00|  0001|     243.75336|POINT (-74.75 84)|
|    84.0|    -74.5|     0|2023-12-27 00:00:00|  0001|     243.74554| POINT (-74.5 84)|
|    84.0|   -74.25|     0|2023-12-27 00:00:00|  0001|     243.73773|POINT (-74.25 84)|
|    84.0|    -74.0|     0|2023-12-27 00:00:00|  0001|     243.72015|   POINT (-74 84)|
|    84.0|   -73.75|     0|2023-12-27 00:00:00|  0001|     243.66351|POINT (-73.75 84)|
|    84.0|    -73.5|     0|2023-12-27 00:00:00|  0001|     243.60687| POINT (-73.5 84)|
|    84.0|   -73.25|     0|2023-12-27 00:00:00|  0001|     243.55023|POINT (-73.25 84)|
|    84.0|    -73.0|     0|2023-

In [23]:
sdf.createOrReplaceTempView("temp_view")

---

### Get Variable Query

In [28]:
# Inputs
min_lat = 70
max_lat = 80
min_lon = -60
max_lon = -50
start_time = "2020-01-01 00:00:00"
end_time = "2021-01-02 00:00:00"


In [33]:
query = f"""
    SELECT time, latitude, longitude, `2m_temperature`
    FROM temp_view
    WHERE latitude BETWEEN {min_lat} AND {max_lat}
        AND longitude BETWEEN {min_lon} AND {max_lon}
    """

In [34]:
result = sedona.sql(query)
result.show()

+-------------------+--------+---------+--------------+
|               time|latitude|longitude|2m_temperature|
+-------------------+--------+---------+--------------+
|2023-12-27 00:00:00|    80.0|    -60.0|     235.80804|
|2023-12-27 00:00:00|    80.0|   -59.75|     236.23187|
|2023-12-27 00:00:00|    80.0|    -59.5|     236.17523|
|2023-12-27 00:00:00|    80.0|   -59.25|     236.12054|
|2023-12-27 00:00:00|    80.0|    -59.0|     236.06586|
|2023-12-27 00:00:00|    80.0|   -58.75|     236.01312|
|2023-12-27 00:00:00|    80.0|    -58.5|     235.96234|
|2023-12-27 00:00:00|    80.0|   -58.25|     235.61859|
|2023-12-27 00:00:00|    80.0|    -58.0|     235.27484|
|2023-12-27 00:00:00|    80.0|   -57.75|     234.93304|
|2023-12-27 00:00:00|    80.0|    -57.5|      234.5932|
|2023-12-27 00:00:00|    80.0|   -57.25|     234.25531|
|2023-12-27 00:00:00|    80.0|    -57.0|     233.83148|
|2023-12-27 00:00:00|    80.0|   -56.75|      233.5346|
|2023-12-27 00:00:00|    80.0|    -56.5|     233

In [35]:
sedona.stop()